## Under pressure

In [1]:
from pathlib import Path
import numpy as np
import re
import itertools

In [2]:
pattern = 'Valve (.+) has flow rate=(\d+); tunnels? leads? to valves? (.+)'
pattern = re.compile(pattern)

In [55]:
valves = Path('valves.txt').read_text().split('\n')

valve_map = {}

for line in valves:
    [key, flow, stations] = re.match(pattern, line).groups()
    
    valve_map[key] = [stations.split(', '), int(flow)]

# Identify valves with non-zero flow and order them from largest to lowest flow
non_zero_valves = [key for key in valve_map if valve_map[key][1] > 0]
non_zero_valves = sorted(non_zero_valves, key = lambda x: valve_map[x][1], reverse = True)

In [56]:
def find_shortest_path(start, end):
    paths = [[start]]
    visited = [start]
    
    if start == end:
        return [start]
    
    while end not in visited:
        new_paths = []
        
        for path in paths:
            last = path[-1]
            routes = valve_map[last][0]
            
            
            for route in routes:
                if route in path:
                    continue
                
                else:
                    if route not in visited:
                        visited += [route]
                    
                    new_path = path + [route]
                
                    new_paths += [new_path]
                    
                    if route == end:
                        return new_path
        
        paths = new_paths
                
    return None

In [58]:
def calculate_path_time(path):
    if len(path) == 0:
        return 0
    else:
        if path[-1] == 'STOP':
            path = path[0:-1]
        time = 0
        for i in range(1, len(path)):
            time += distances[path[i-1], path[i]]
            
        return time

In [59]:
locations = ['AA'] + non_zero_valves

distances = {}

for start in locations:
    for end in locations:
        distances[start, end] = len(find_shortest_path(start, end))-1

In [60]:
def calculate_total_pressure(path, time_limit = 30):
    if path[-1] == 'STOP':
        path = path[0:-1]
        
    times = np.zeros(len(path), dtype = 'int')
    for i in range(1, len(path)):
        times[i] = times[i-1] + distances[path[i-1], path[i]] + 1
        
    pressures = np.array([valve_map[valve][1] for valve in path])
    
    time_remaining = time_limit - times
    
    time_remaining[time_remaining < 0] = 0
    
    pressure_released = np.sum(time_remaining * pressures)
        
    return pressure_released
        

In [61]:
def max_pressure(path, valves = non_zero_valves, time_limit = 30, distances = distances):
    if path[-1] == 'STOP':
        path = path[0:-1]
    
    base_pressure = calculate_total_pressure(path, time_limit)
    
    time_remaining = time_limit - calculate_path_time(path) - len(path) + 1
    
    last_valve = path[-1]
    
    remaining_valves = np.setdiff1d(valves, path)
    
    additional_pressure = sum([valve_map[valve][1]*(time_remaining - distances[last_valve, valve] - 1) 
                               for valve in remaining_valves if distances[last_valve, valve] < time_remaining-1])
    
    return base_pressure + additional_pressure

In [105]:
start = 'AA'

num_nzv = len(non_zero_valves)

paths = [[start]]

final_paths = []

time_limit = 26

for i in range(num_nzv):
    new_paths = []
    
    best_so_far = max([calculate_total_pressure(path, time_limit) for path in paths])

    for path in paths:
        
        if max_pressure(path, non_zero_valves, time_limit) < best_so_far:
            continue
        
        if path[-1] == 'STOP':
            final_paths += [path]
            continue
        
        elif calculate_path_time(path) > time_limit - len(path):
            final_paths += [path + ['STOP']]
            continue
        
        remaining_valves = np.setdiff1d(non_zero_valves, path)
        
        for valve in remaining_valves:
            new_path = path + [valve]
            
            new_paths += [new_path]
        
    paths = new_paths
    
    if len(new_paths) == 0:
        break
    
final_paths += paths

In [104]:
max([calculate_total_pressure(path, 26) for path in final_paths])

1205

## Part 2: This is our last dance

In [107]:
def best_pressure(valves = non_zero_valves, time_limit = 26):

    start = 'AA'

    paths = [[start]]
    
    final_paths = []

    for i in range(len(valves)):
        new_paths = []

        best_so_far = max([calculate_total_pressure(path, time_limit) for path in paths])

        for path in paths:

            if max_pressure(path, valves, time_limit) < best_so_far:
                continue

            if path[-1] == 'STOP':
                final_paths += [path]
                continue

            elif calculate_path_time(path) > time_limit - len(path):
                final_paths += [path + ['STOP']]
                continue

            remaining_valves = np.setdiff1d(valves, path)

            for valve in remaining_valves:
                new_path = path + [valve]

                new_paths += [new_path]

        

        paths = new_paths
        
        if len(new_paths) == 0:
            break
        
    final_paths += paths

    return max([calculate_total_pressure(path, time_limit) for path in final_paths])

In [109]:
best_combined_pressure = 0

for i in range(1, len(non_zero_valves)//2+1):
    print(i)
    my_valve_sets = itertools.combinations(non_zero_valves, i)
    
    for my_valves in my_valve_sets:
        elephant_valves = np.setdiff1d(non_zero_valves, my_valves)
        
        best_combined_pressure = max(best_combined_pressure, 
                                     best_pressure(my_valves, 26) + best_pressure(elephant_valves, 26))

1
2
3
4
5
6
7


In [110]:
best_combined_pressure

2343